 #### About
 1. CycleGAN attempts to learn a mapping from one dataset to another like a dataset of plants to dataset of faces.
 2. It does this with two generators G and F and two discriminators Dx and Dy.
 3. G attempts to turn dataset A into dataset B and F attempts to turn dataset B into dataset A.
 4. Dx is used to differentiate real images sampled from Dataset A and images generated by generator F. F is updated to get better at fooling Dx
 5. Dy is used to differentiate real images sampled from Dataset B and images generated by generator G. G is updated to get better at fooling Dy.
 6. Introduction of cycle loss is made to ensure an image that is yielded by generator can be mapped back to original image by other generator i.e F(G(x)) - x and G(F(y)) - y. Two cycles of Generators.
 7. Introduction of identity loss is made to help preserve tint, contrast and color shade. It ensures that when an image of target class is given, Generator yields the same image. F(x) - x and G(y) - y.
 8. A series of conv layers and residual layers are used in generator to map one image to another.
 9. PatchGAN architecture is used to classify images via Discriminator.
 10. Reflection padding is used in architecture instead of zero padding to reduce artifacts.
 11. The generator consists of encoder and decoder. It downsamples or encode the input image then interpret the encoding with series of residual blocks having skip connections. After this, Upsampling is being done to decode the representation to the size of the fake image.
 12. Instance normalisation is used instead of batch normalisation which results in better image generated by operating across image channels.
 13. Discriminator uses a PatchGAN in modification to the regular GAN architecture. The regular GAN maps an input image to single scalar output in range of 0 to 1 indicating the image being real or fake via probability. Whereas PatchGAN provides a matrix as output where each element of the matrix signifies whether its corresponding patch is real or fake. The receptive field of discriminator is 70*70 in Cycle GAN.
 ![cycle_gan](cycle_gan.png)
 14. To reduce model oscillation, Replay buffer method is implemented. The discriminator is updated using a history of generated images rather than the ones produced by the latest generators. An image buffer to store previous created images is stored.